In [7]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [21]:
# Constants
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "/Users/hongtan/Desktop/auto-translator/cmn-eng/cmn.txt"

In [14]:
# Vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [16]:
# 
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
            
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [17]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [18]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 73
Number of unique output tokens: 2470
Max sequence length for inputs: 26
Max sequence length for outputs: 22


In [19]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [20]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [22]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

Metal device set to: Apple M1 Pro


In [23]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

In [24]:
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Epoch 1/100


2023-05-23 14:21:12.281980: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


125/125 [==============================] - 9s 48ms/step - loss: 2.1034 - accuracy: 0.6656 - val_loss: 2.3580 - val_accuracy: 0.6047
Epoch 2/100
125/125 [==============================] - 5s 36ms/step - loss: 1.8175 - accuracy: 0.7056 - val_loss: 2.0974 - val_accuracy: 0.6806
Epoch 3/100
125/125 [==============================] - 4s 35ms/step - loss: 1.6197 - accuracy: 0.7423 - val_loss: 1.9780 - val_accuracy: 0.6884
Epoch 4/100
125/125 [==============================] - 4s 35ms/step - loss: 1.5109 - accuracy: 0.7558 - val_loss: 1.8931 - val_accuracy: 0.7014
Epoch 5/100
125/125 [==============================] - 4s 35ms/step - loss: 1.4253 - accuracy: 0.7679 - val_loss: 1.8184 - val_accuracy: 0.7128
Epoch 6/100
125/125 [==============================] - 4s 36ms/step - loss: 1.3550 - accuracy: 0.7763 - val_loss: 1.7796 - val_accuracy: 0.7158
Epoch 7/100
125/125 [==============================] - 5s 37ms/step - loss: 1.2885 - accuracy: 0.7846 - val_loss: 1.7317 - val_accuracy: 0.7245
Epoc

125/125 [==============================] - 5s 38ms/step - loss: 0.3527 - accuracy: 0.9233 - val_loss: 1.5744 - val_accuracy: 0.7720
Epoch 58/100
125/125 [==============================] - 5s 38ms/step - loss: 0.3470 - accuracy: 0.9241 - val_loss: 1.5701 - val_accuracy: 0.7733
Epoch 59/100
125/125 [==============================] - 5s 38ms/step - loss: 0.3417 - accuracy: 0.9252 - val_loss: 1.5708 - val_accuracy: 0.7743
Epoch 60/100
125/125 [==============================] - 5s 38ms/step - loss: 0.3345 - accuracy: 0.9264 - val_loss: 1.5746 - val_accuracy: 0.7736
Epoch 61/100
125/125 [==============================] - 5s 37ms/step - loss: 0.3295 - accuracy: 0.9277 - val_loss: 1.6003 - val_accuracy: 0.7732
Epoch 62/100
125/125 [==============================] - 5s 38ms/step - loss: 0.3236 - accuracy: 0.9286 - val_loss: 1.5947 - val_accuracy: 0.7715
Epoch 63/100
125/125 [==============================] - 5s 38ms/step - loss: 0.3191 - accuracy: 0.9296 - val_loss: 1.5950 - val_accuracy: 0.774

In [25]:
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [26]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

1/1 [==============================] - 0s 14ms/step
-
Input sentence: Hi.
Decoded sentence: 嗨。

1/1 [==============================] - 0s 14ms/step
-
Input sentence: Hi.
Decoded sentence: 嗨。

1/1 [==============================] - 0s 14ms/step
-
Input sentence: Run.
Decoded sentence: 你用跑的。

1/1 [==============================] - 0s 14ms/step
-
Input sentence: Stop!
Decoded sentence: 住手！

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Wait!
Decoded sentence: 等等！

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Wait!
Decoded sentence: 等等！

1/1 [==============================] - 0s 14ms/step
-
Input sentence: Begin.
Decoded sentence: 开始！

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Hello!
Decoded sentence: 你好。

1/1 [==============================] - 0s 14ms/step
-
Input sentence: I try.
Decoded sentence: 我试试。

1/1 [==============================] - 0s 15ms/step
-
Input sentence: I won!
Decoded sentence: 我不是學生。

1/1 [==

In [1]:
# Preprocess data
with open('/Users/hongtan/Desktop/auto-translator/cmn-eng/cmn.txt', 'r') as f:
    translations = []
    lines = f.readlines()
    for line in lines:
        translations.append(line.split('\t')[:2]) # Get English and Chinese words

In [2]:
translations[:5]

[['Hi.', '嗨。'],
 ['Hi.', '你好。'],
 ['Run.', '你用跑的。'],
 ['Stop!', '住手！'],
 ['Wait!', '等等！']]

In [3]:
# Get list of English words and punctuations
s = set()
for words in translations:
    word = words[0].split()
    for w in word:
        w = w.lower()
        if ord(w[-1]) < 97 or ord(w[-1]) > 122:
            s.add(w[-1])
            w = w.replace(w[-1],'')
        s.add(w)
print(len(s))

7516


In [6]:
words = list(s)
words[5:10]

['allied', 'soil', 'only', 'fainted', 'authority']

In [ ]:
tokens = {}
for i,x in enumerate(words):
    tokens[x] = i

In [ ]:
s = "This is a short sentence ."
print("Input: ", s)
s = s.lower().split()
print("Output: ", end='')
for i in s:
    print(tokens[i], end=' ')